# Set up paths

In [1]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib_r42',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

#Set the vis device
setHook(packageEvent("grDevices", "onLoad"),
function(...) grDevices::X11.options(type='cairo'))
options(device='x11')

[1] "/v-data2/matt_davenport/HaNa_RE/HaNa_Reanalysis/hana_reanalysis_lib_r42"


[1] "/v-data2/matt_davenport/HaNa_RE/HaNa_Reanalysis/hana_reanalysis_lib_r42"
[2] "/home/mdavenport/bin/R4.2.1/R-4.2.1/library"

# 4 - Defining Continuous Module Membership

In [2]:
library(tidyverse)
library(WGCNA)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [3]:
net <- readRDS('3_wgcna_net_associatedToTraits.rds')
names(net)

[1] "colors"         "unmergedColors" "cutreeLabels"   "MEs"           
[5] "MEsOK"          "input_data"     "MEG_res_tbls"   "node_me_tbls"

## First, lets wrangle that colors thing into a nice tibble.

In [4]:
gene_tbl <- net$colors %>% as.data.frame
colnames(gene_tbl) <- 'me_assigned'
gene_tbl$gene_id<- row.names(gene_tbl)
gene_tbl <- gene_tbl %>% as_tibble %>% mutate(me_assigned = recode_factor(as.factor(me_assigned),
                                                        '1'='A',
                                                        '2'='B',
                                                        '3'='C',
                                                        '4'='D',
                                                        '5'='E',
                                                        '6'='F',

                                                        '7'='G',
                                                        '8'='H',
                                                        '9'='I',
                                                        '10'='J',
                                                        '11'='K',

                                                        '12'='L',
                                                        '13'='X'))
head(gene_tbl)
levels(gene_tbl$me_assigned)

me_assigned,gene_id
<fct>,<chr>
X,A1CF
X,A2M
J,A4GNT
D,AAAS
A,AACS
X,AADAC


[1] "A" "B" "C" "D" "E" "F" "G" "H" "I" "J" "K" "L" "X"

## Now get the expression of genes (row samples cols genes and MEGs)

In [5]:
gene_mod_cor_tbl <- net$input_data$fpkm %>% as_tibble(rownames='sample_id') 
head(gene_mod_cor_tbl)

sample_id,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAGAB,AAK1,⋯,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fe_an_1,0.00000000,7.093980,0.00000000,7.692193,49.32907,0.03223515,0.6634673,31.15091,39.58131,⋯,12.36159,2.094811,11.435813,4.783891,2.111552,3.601635,50.19418,8.886919,6.180941,8.451389
fe_an_2,0.04557418,4.603331,0.00000000,7.222035,44.81534,0.00000000,0.3810742,33.07874,35.77192,⋯,19.13082,1.606932,7.308235,5.893162,3.651458,2.799674,60.55626,8.327203,5.846237,6.702416
fe_an_3,0.02213098,6.060817,0.00000000,6.012078,37.57134,0.00000000,1.1488581,32.89971,43.11340,⋯,17.90711,2.392109,7.846031,4.413440,3.580818,2.817938,52.33897,8.748009,8.576732,8.207267
fe_ax_1,0.00000000,9.463016,0.06496191,4.592011,22.15344,0.03798045,1.0960165,26.90928,29.74793,⋯,14.90511,3.796637,15.475760,6.061926,3.309983,3.197201,16.52732,7.346755,5.679001,10.680909
fe_ax_2,0.04701613,6.334622,0.15844851,9.687336,19.60612,0.04631903,0.3669225,28.41250,35.30265,⋯,17.87317,3.396628,11.164825,5.079860,2.392111,3.205963,14.71874,6.689560,4.276965,8.967043
fe_ax_3,0.00000000,5.639964,0.00000000,6.831030,16.69971,0.00000000,1.3223168,35.06826,32.36720,⋯,11.81573,5.945649,8.521584,2.561196,3.114575,2.933240,12.00139,5.899499,6.239419,12.185660


In [6]:
mes <- net$MEs
names(mes)

[1] "ME2"       "ME8"       "ME4"       "ME11"      "ME1"       "ME14"     
 [7] "ME5"       "ME12"      "ME6"       "ME9"       "ME3"       "ME10"     
[13] "sample_id"

In [7]:
names(mes) = c('B', 'G', 'D', 'J', 'A', 'L', 'E', 'K', 'F', 'H', 'C', 'I', 'sample_id')
Mes <- mes %>% as_tibble(rownames='sample_id') 
gene_mod_cor_tbl <- left_join(mes,gene_mod_cor_tbl)
head(gene_mod_cor_tbl)

Joining, by = "sample_id"


,B,G,D,J,A,L,E,K,F,H,⋯,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.13228979,-0.027382991,-0.037637380,-0.015783669,-0.05466811,0.223235091,-0.10033038,-0.04839298,0.01062152,-0.034954535,⋯,12.36159,2.094811,11.435813,4.783891,2.111552,3.601635,50.19418,8.886919,6.180941,8.451389
2,0.15480032,-0.028171498,-0.002373231,0.256800784,-0.08578654,-0.038752513,-0.10197053,-0.04006914,-0.01404994,-0.022238371,⋯,19.13082,1.606932,7.308235,5.893162,3.651458,2.799674,60.55626,8.327203,5.846237,6.702416
3,0.11725144,0.005571085,-0.034747519,-0.023217746,-0.02021596,-0.020950488,-0.09059430,-0.01980841,0.04255413,-0.179675809,⋯,17.90711,2.392109,7.846031,4.413440,3.580818,2.817938,52.33897,8.748009,8.576732,8.207267
4,0.01566468,-0.040542373,-0.206990365,0.027966101,0.19810091,0.424405897,-0.08530025,-0.04868079,-0.04309892,0.162617704,⋯,14.90511,3.796637,15.475760,6.061926,3.309983,3.197201,16.52732,7.346755,5.679001,10.680909
5,-0.04173157,-0.053452053,-0.013933284,0.397512233,0.15094658,0.015225273,-0.10250359,-0.03704185,-0.05772286,0.005850349,⋯,17.87317,3.396628,11.164825,5.079860,2.392111,3.205963,14.71874,6.689560,4.276965,8.967043
6,-0.06386408,-0.039814282,-0.087049311,0.006507481,0.22478553,0.008869193,-0.08940201,-0.02379018,-0.03756456,0.136546573,⋯,11.81573,5.945649,8.521584,2.561196,3.114575,2.933240,12.00139,5.899499,6.239419,12.185660


## Now I need to correlate each of the genes to each of the MEGs across samples
## I want a matrix with cols = genes, rows = MEGs, value is results of WGCNA cor 

In [8]:
megs <-c('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L')
res <- gene_mod_cor_tbl %>% select(one_of(megs)) %>%
    lapply(function(meg){
        res <- gene_mod_cor_tbl %>% select(-one_of(c(megs,'sample_id'))) %>%
            lapply(function(gene){
                return(cor(gene,meg))
            })
        return(res)
    })

In [9]:
res2 <- res %>% as_tibble 


In [10]:
res3 <- res2 %>% mutate(gene_id = names(res2$A)) %>% mutate_at(megs,as.numeric) %>% left_join(gene_tbl)
head(res3)

Joining, by = "gene_id"


A,B,C,D,E,F,G,H,I,J,K,L,gene_id,me_assigned
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
-0.03370932,0.18224771,-0.09239870,-0.15846667,0.17859488,0.001863475,0.12842694,-0.07431761,-0.07838895,-0.04283075,-0.11005667,-0.25708316,A1CF,X
0.02510845,-0.36403524,0.30267066,-0.08856526,0.44178052,0.037101574,-0.05202142,0.13264074,0.33842222,-0.05095856,-0.15834949,0.30518858,A2M,X
0.18255477,0.10507450,-0.10350080,-0.06411247,-0.08134665,-0.112865812,0.10047994,-0.04726497,-0.11964807,0.50213499,0.05178730,0.09712287,A4GNT,J
-0.17604402,-0.36389087,0.33743185,0.76031742,0.05085780,-0.326007923,-0.07671473,-0.15493875,-0.06018514,0.15980198,0.14182718,-0.10599198,AAAS,D
-0.72859604,0.26244931,0.29321445,0.02052947,0.04168985,0.032170986,0.27472125,-0.06501581,-0.13804126,-0.04817724,0.07641915,-0.04250240,AACS,A
-0.07273316,0.09715425,0.02042227,0.07404382,-0.05979183,-0.064351647,0.10828197,-0.02043833,-0.14172340,-0.13060679,-0.07616428,-0.02772431,AADAC,X


In [12]:
saveRDS(res3,'4_mod_membership_all_genes_broken_out.rds')

In [13]:
sessionInfo()

R version 4.2.1 (2022-06-23)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.1 LTS

Matrix products: default
BLAS:   /home/mdavenport/bin/R4.2.1/R-4.2.1/lib/libRblas.so
LAPACK: /home/mdavenport/bin/R4.2.1/R-4.2.1/lib/libRlapack.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] WGCNA_1.71            fastcluster_1.2.3     dynamicTreeCut_1.63-1
 [4] forcats_0.5.2         stringr_1.4.1         dplyr_1.0.10         
 [7] purrr_0.3.5           readr_2.1.3           tidyr_1.2.1          
[10] tibble_3.1.8          ggplot2_3.3.6         tidyverse_1.3.2      

loaded via a namespace (and not att